#### Generating a df with the raw data, and from that, a df with only the income, and a df with only the expenses

In [17]:
import pandas as pd

df_data = pd.read_csv("demo_raw_data_personal_expenses.csv", delimiter=";")

df_data["date"] = pd.to_datetime (df_data["date"], format= "%d/%m/%Y")

df_data.to_csv("exported_dataframes/df_data.csv")

#df_data

# DF FOR INCOME AND EXPENSES

df_income = df_data.copy()
df_income = df_income [df_income ["main_category"] == "Income"] 

df_income.to_csv("exported_dataframes/df_income.csv")

#df_income

df_expenses = df_data.copy()
df_expenses = df_expenses [df_expenses ["main_category"] != "Income"] 

list_essentials = ["Car", "Groceries", "Health", "Home Expenses", "Public Transports", "Taxes", "Telecom"]
df_expenses.loc [df_expenses["main_category"].isin(list_essentials), "essential"] = "Essentials"
df_expenses.loc [~df_expenses["main_category"].isin(list_essentials), "essential"] = "Non-Essentials"

df_expenses.to_csv("exported_dataframes/df_expenses.csv")

df_expenses

,id_expense,value,date,account_number,category_id,main_category,secondary_category,essential
0,1,-1200.00,2023-01-01,1111,8,Home Expenses,Rent,Essentials
1,2,-40.00,2023-01-02,2222,19,Health,Gym,Essentials
2,3,-100.00,2023-01-04,3333,20,Health,Health Insurance,Essentials
4,5,-1200.00,2023-02-05,2222,8,Home Expenses,Rent,Essentials
5,6,-40.00,2023-02-01,1111,19,Health,Gym,Essentials
...,...,...,...,...,...,...,...,...
1995,1996,-97.46,2024-05-31,2222,27,Clothing,Shoes,Non-Essentials
1996,1997,-53.22,2025-04-02,1111,3,Purchases,Home Items,Non-Essentials
1997,1998,-551.20,2023-06-17,2222,9,Holidays,Airplane,Non-Essentials
1998,1999,-117.74,2024-06-11,1111,16,Leisure,Restaurant,Non-Essentials


______________

____

____________________


_________________________


#### Generating the Dataframe with the evolution

In [40]:
df_evolution = df_data.copy()
df_evolution = df_evolution.sort_values("date")

#df_evolution ["current_balance"] = df_evolution.groupby ("account_number") ["value"].cumsum()
df_evolution ["balance_account_1111"] = (df_evolution [df_evolution ["account_number"] == 1111]) ["value"].cumsum()
df_evolution ["balance_account_2222"] = (df_evolution [df_evolution ["account_number"] == 2222]) ["value"].cumsum()
df_evolution ["balance_account_3333"] = (df_evolution [df_evolution ["account_number"] == 3333]) ["value"].cumsum()
# fill the NaN
df_evolution [["balance_account_1111", "balance_account_2222", "balance_account_3333"]] = df_evolution[["balance_account_1111", "balance_account_2222", "balance_account_3333"]].ffill().fillna(0)

df_evolution ["total_balance"] = df_evolution ["balance_account_1111"] + df_evolution ["balance_account_2222"] + df_evolution ["balance_account_3333"]

df_evolution.to_csv("exported_dataframes/df_evolution.csv")

#df_evolution

#### Daily datasets

In [45]:

df_evolution_daily = df_evolution.copy()
df_evolution_daily = df_evolution_daily [["date", "balance_account_1111", "balance_account_2222", "balance_account_3333", "total_balance"]].groupby ("date") [["balance_account_1111", "balance_account_2222", "balance_account_3333", "total_balance"]].last().reset_index()

df_evo_1111 = df_evolution_daily[["date", "balance_account_1111"]]
df_evo_1111 = df_evo_1111.rename (columns = {"balance_account_1111": "balance"})
df_evo_1111 ["account"] = "Account #1111"
df_evo_2222 = df_evolution_daily[["date", "balance_account_2222"]]
df_evo_2222 = df_evo_2222.rename (columns = {"balance_account_2222": "balance"})
df_evo_2222 ["account"] = "Account #2222"
df_evo_3333 = df_evolution_daily[["date", "balance_account_3333"]]
df_evo_3333 = df_evo_3333.rename (columns = {"balance_account_3333": "balance"})
df_evo_3333 ["account"] = "Account #3333"
df_evolution_daily_plot = pd.concat( [df_evo_1111, df_evo_2222, df_evo_3333] , ignore_index = True)

df_evolution_daily.to_csv("exported_dataframes/df_evolution_daily.csv")
df_evolution_daily_plot.to_csv("exported_dataframes/df_evolution_daily_plot.csv")

df_evolution_daily
#df_evolution_daily_plot

,date,balance_account_1111,balance_account_2222,balance_account_3333,total_balance
0,2023-01-01,-1200.00,-97.55,0.00,-1297.55
1,2023-01-02,-1338.06,-137.55,0.00,-1475.61
2,2023-01-03,10661.94,-234.81,0.00,10427.13
3,2023-01-04,10661.94,-285.77,-140.59,10235.58
4,2023-01-05,10661.94,-376.42,-184.66,10100.86
...,...,...,...,...,...
896,2025-11-21,45929.22,15121.38,44253.12,105303.72
897,2025-11-23,45828.55,15109.31,44253.12,105190.98
898,2025-11-24,45828.55,15034.41,43778.89,104641.85
899,2025-11-25,45685.63,14879.71,43778.89,104344.23


#### Monthly datasets

In [ ]:
df_evolution_month = df_evolution.copy()
df_evolution_month = df_evolution_month [["date", "balance_account_1111", "balance_account_2222", "balance_account_3333", "total_balance"]].groupby ("date") [["balance_account_1111", "balance_account_2222", "balance_account_3333", "total_balance"]].last()
df_evolution_month = df_evolution_month.resample('ME').last()

df_evolution_month = df_evolution_month.reset_index()
df_evo_1111 = df_evolution_month[["date", "balance_account_1111"]]
df_evo_1111 = df_evo_1111.rename (columns = {"balance_account_1111": "balance"})
df_evo_1111 ["account"] = "Account #1111"
df_evo_2222 = df_evolution_month[["date", "balance_account_2222"]]
df_evo_2222 = df_evo_2222.rename (columns = {"balance_account_2222": "balance"})
df_evo_2222 ["account"] = "Account #2222"
df_evo_3333 = df_evolution_month[["date", "balance_account_3333"]]
df_evo_3333 = df_evo_3333.rename (columns = {"balance_account_3333": "balance"})
df_evo_3333 ["account"] = "Account #3333"
df_evolution_month_plot = pd.concat( [df_evo_1111, df_evo_2222, df_evo_3333] , ignore_index = True)

df_evolution_month.to_csv("exported_dataframes/df_evolution_month.csv")
df_evolution_month_plot.to_csv("exported_dataframes/df_evolution_month_plot.csv")

df_evolution_month
df_evolution_month_plot


,date,balance,account
0,2023-01-31,7708.61,Account #1111
1,2023-02-28,4221.58,Account #1111
2,2023-03-31,14397.05,Account #1111
3,2023-04-30,11093.34,Account #1111
4,2023-05-31,9928.17,Account #1111
...,...,...,...
100,2025-07-31,31127.71,Account #3333
101,2025-08-31,27572.02,Account #3333
102,2025-09-30,37084.65,Account #3333
103,2025-10-31,34925.36,Account #3333


#### Yearly datasets

In [52]:
df_evolution_year = df_evolution.copy()
df_evolution_year = df_evolution_year [["date", "balance_account_1111", "balance_account_2222", "balance_account_3333", "total_balance"]].groupby ("date") [["balance_account_1111", "balance_account_2222", "balance_account_3333", "total_balance"]].last()
df_evolution_year = df_evolution_year.resample('YE').last()

df_evolution_year = df_evolution_year.reset_index()
df_evo_1111 = df_evolution_year[["date", "balance_account_1111"]]
df_evo_1111 = df_evo_1111.rename (columns = {"balance_account_1111": "balance"})
df_evo_1111 ["account"] = "Account #1111"
df_evo_2222 = df_evolution_year[["date", "balance_account_2222"]]
df_evo_2222 = df_evo_2222.rename (columns = {"balance_account_2222": "balance"})
df_evo_2222 ["account"] = "Account #2222"
df_evo_3333 = df_evolution_year[["date", "balance_account_3333"]]
df_evo_3333 = df_evo_3333.rename (columns = {"balance_account_3333": "balance"})
df_evo_3333 ["account"] = "Account #3333"
df_evolution_year_plot = pd.concat( [df_evo_1111, df_evo_2222, df_evo_3333] , ignore_index = True)

df_evolution_year.to_csv("exported_dataframes/df_evolution_year.csv")
df_evolution_year_plot.to_csv("exported_dataframes/df_evolution_year_plot.csv")

df_evolution_year
df_evolution_year_plot


,date,balance,account
0,2023-12-31,16487.40,Account #1111
1,2024-12-31,-6794.41,Account #1111
2,2025-12-31,45685.63,Account #1111
3,2023-12-31,19328.61,Account #2222
4,2024-12-31,33707.52,Account #2222
5,2025-12-31,14879.71,Account #2222
6,2023-12-31,3681.04,Account #3333
7,2024-12-31,41184.98,Account #3333
8,2025-12-31,43681.57,Account #3333
